In [1]:
from pycorese.api import CoreseAPI

python_to_java_bridge = 'py4j'
corese = CoreseAPI(java_bridge=python_to_java_bridge)
_ = corese.loadCorese()

2025-04-13 09:43:14,578 - INFO - Py4J: Loading CORESE...
2025-04-13 09:43:15,203 - INFO - Py4J: CORESE is loaded


In [2]:
g = corese.loadRDF("../data/graph_data.rdf")

In [3]:
# Simple first 5 triples from the whole graph
res = corese.sparqlSelect(g, "SELECT * WHERE { ?s ?p ?o } LIMIT 5", return_dataframe=True)
res

,s,p,o
0,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#country
1,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#managedBy
2,_:b2,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#manages...
3,_:b3,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#item
4,_:b4,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#country


In [4]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
1,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
2,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
3,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
4,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
5,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
6,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL
7,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE


In [ ]:
# Run the rules engine
corese.resetRuleEngine(g)
corese.loadRuleEngine(g, profile='owlrl')
corese.loadRuleEngine(g, profile='owlrl_lite')
corese.loadRuleEngine(g, profile='owlrl_ext')
corese.loadRuleEngine(g, profile='rdfs')

In [13]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#KITCHEN
1,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
2,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
3,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
4,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
...,...,...,...
61,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
62,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
63,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
64,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL


In [14]:
# Find all the records that a specific user may have access to.

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select ?record where {{
?department a :Department .
?department :hasItem ?item .
{{
data:{USER} :hasAccess ?country, ?department ; a :User .
?country a :Country .
?record a :Record; :country ?country; :item ?item.}}
union {{
data:{USER} a :Manager .
data:{USER} :managesDepartment ?department .
?record a :Record; :item ?item.
}}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record


In [ ]:
query = """
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select * where {
data:SalesRep_0 a :User ; :hasAccess ?country, ?department .
?department a :Department .
?country a :Country .
?department :hasItem ?item .
?record a :Record; :country ?country; :item ?item.
}
limit 10
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,user,country,department,item,record
0,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#486591,local://security.permissions.net/entities#4865...
1,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#470757,local://security.permissions.net/entities#4707...
2,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#477120,local://security.permissions.net/entities#4771...
3,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#489195,local://security.permissions.net/entities#4891...
4,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#464898,local://security.permissions.net/entities#4648...
5,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#470883,local://security.permissions.net/entities#4708...
6,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#476679,local://security.permissions.net/entities#4766...
7,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#489426,local://security.permissions.net/entities#4894...
8,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#486633,local://security.permissions.net/entities#4866...
9,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Israel,local://security.permissions.net/entities#KITCHEN,local://security.permissions.net/entities#441735,local://security.permissions.net/entities#4417...
